## 處理缺失資料 (null, NaN, NA)
### 對於缺失資料慣例的取捨

* 使用遮罩(mask) : 布林陣列、資料中的其中一個位元
* 哨兵值(sentinel value) : -99999、Nan(Not a Number)是IEEE浮點數規中的一個特殊值

### None : Python 的缺失資料
* None 是一個 Python的物件，不能隨意使用在 Numpy / Pandas 的陣列中
* 資料型態是 object

In [2]:
import numpy as np
import pandas as pd

vals1 = np.array([1,None,3,4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object','int']:
    print("dtype =",dtype)
    %timeit np.arange(1E6,dtype=dtype).sum()
    print()

dtype = object
69.4 ms ± 624 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.84 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



In [4]:
#vals1.sum() # 會有錯

### NaN : 缺失的數值資料
* 特殊的浮點數

In [5]:
vals2 = np.array([1,np.nan,3,4])
vals2,vals2.dtype

(array([ 1., nan,  3.,  4.]), dtype('float64'))

In [6]:
# NaN 跟任何數字運算都會是 NaN
1 + np.nan, 0 * np.nan, 0/np.nan, vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan, nan, nan, nan)

In [7]:
# 忽略缺失值
np.nansum(vals2),np.nanmax(vals2),np.nanmin(vals2)

(8.0, 4.0, 1.0)

### 在 Pandas 中的 NaN 和 None
NaN 和 None 都有它們存在，而Pandas則是交換著使用他們，在他們之間適當地轉換:

In [8]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

對於那些沒有哨兵值的型態來說，Pandas會在NA值出現時自動轉型

In [9]:
# 放進整數陣列轉浮點數
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [10]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

### 在Null值上操作 (isunll、notnull、fillna)
偵測、移除、取代空值

In [11]:
# 偵測空值
data = pd.Series([1,np.nan,'hello',None])
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [13]:
# 搭配遮罩
data[data.notnull()]


0        1
2    hello
dtype: object

In [15]:
# 拋棄空值
data.dropna()

0        1
2    hello
dtype: object

In [17]:
df = pd.DataFrame([[1,np.nan,2],
                  [2,3,5],
                  [np.nan,4,6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [23]:
# 有 NaN的row 刪掉
df.dropna(axis=0)

,0,1,2
1,2.0,3.0,5


In [24]:
# 有 NaN的column刪掉
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [25]:
3 # 過濾允許幾個空值 (how、thresh)
df[3] = np.nan 
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [27]:
# 欄全是空值才刪掉
df.dropna(axis=1,how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [29]:
# 至少row要有3個非空值資料
df.dropna(axis=0,thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [31]:
# 填入空值
data = pd.Series([1,np.nan,2,None,3],index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [32]:
# 空塞0
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [33]:
# 填前一個值
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [34]:
# 填後一個值
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [35]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [36]:
df.fillna(method='ffill',axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


## 階層式索引( Hierarchical Indexing )
透過在一個索引中合併多個索引層級來進行

In [37]:
 import numpy as np
 import pandas as pd

### 一個多重索引的Series
#### 不好的方法

In [38]:
# 2個不同年度的州資料
index = [('California',2000),('California',2010),
         ('New York',2000),('New York',2010),
         ('Texas',2000),('Texas',2010)]
population = [33871559,8885522,
              17896542,4549132,
              20587842,6445123]
pop = pd.Series(population,index=index)
pop

(California, 2000)    33871559
(California, 2010)     8885522
(New York, 2000)      17896542
(New York, 2010)       4549132
(Texas, 2000)         20587842
(Texas, 2010)          6445123
dtype: int64

In [39]:
pop[('California',2000):('Texas',2000)]

(California, 2000)    33871559
(California, 2010)     8885522
(New York, 2000)      17896542
(New York, 2010)       4549132
(Texas, 2000)         20587842
dtype: int64

In [42]:
# 檢索特定年分，麻煩，速度也慢
pop[[i for i in pop.index if i[1]==2010 ]]

(California, 2010)    8885522
(New York, 2010)      4549132
(Texas, 2010)         6445123
dtype: int64

#### 比較好的方法 
tuple 是多重索引退化版本、MultiIndex提供了我們要的型態

In [43]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [44]:
# 第一欄空白處表示跟上方相同
pop = pop.reindex(index)
pop

California  2000    33871559
            2010     8885522
New York    2000    17896542
            2010     4549132
Texas       2000    20587842
            2010     6445123
dtype: int64

In [46]:
# 切片
pop[:,2010]

California    8885522
New York      4549132
Texas         6445123
dtype: int64

### 多重索引當作是額外的維度


In [51]:
# 將多重索引的Series轉乘DataFrame
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871559,8885522
New York,17896542,4549132
Texas,20587842,6445123


In [53]:
pop_df.stack()

California  2000    33871559
            2010     8885522
New York    2000    17896542
            2010     4549132
Texas       2000    20587842
            2010     6445123
dtype: int64

In [54]:
# 加欄位
pop_df = pd.DataFrame({'total':pop,
                       'under18':[9267089,9284094,
                                  4687374,4318033,
                                  5906301,6879014]})
pop_df

total  under18
California 2000  33871559  9267089
           2010   8885522  9284094
New York   2000  17896542  4687374
           2010   4549132  4318033
Texas      2000  20587842  5906301
           2010   6445123  6879014

In [55]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273595,1.044856
New York,0.261915,0.949199
Texas,0.286883,1.067321


In [57]:
# 建立 Multilindex建立方法
df = pd.DataFrame(np.random.rand(4,2),index=[['a','a','b','b'],[1,2,1,2]],columns=['data1','data2'])
df

data1     data2
a 1  0.398285  0.915257
  2  0.654155  0.888259
b 1  0.307958  0.468266
  2  0.097670  0.389661

In [58]:
data = {('California',2000):33871648,
        ('California',2010):37253956,
        ('Texas',2000):20851820,
        ('Texas',2010):25145561,
        ('New York',2000):18976457,
        ('New York',2000):19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    19378102
dtype: int64

In [59]:
# 明確指定 Multindex 建構子
pd.MultiIndex.from_arrays([['a','a','b','b'],[1,2,1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [60]:
pd.MultiIndex.from_tuples([('a',1),('a',2),('b',1),('b',2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [61]:
#笛卡兒積(Cartesian product)
pd.MultiIndex.from_product([['a','b'],[1,2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### 多重索引的階層名稱

In [73]:
pop.index.names = ['state','year']
pop

state       year
California  2000    33871559
            2010     8885522
New York    2000    17896542
            2010     4549132
Texas       2000    20587842
            2010     6445123
dtype: int64

### 欄的Multilndex

In [84]:
# 階層式的索引和欄
index = pd.MultiIndex.from_product([[2013,2014],[1,2]],names=['year','visit'])
columns = pd.MultiIndex.from_product([['Bob','Guido','Sue'],['HR','Temp']],names=['subject','type'])
#仿製一些資料
data = np.round(np.random.randn(4,6),1) # round取小數第一位
data[:,::2] *= 10 # 欄 0、2、4 * 10
data += 37
# 建立 DataFrame
health_data = pd.DataFrame(data,index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  36.8  53.0  36.3  37.0  36.9
     2      50.0  38.1  21.0  35.7  48.0  37.0
2014 1      29.0  37.9  42.0  36.2  51.0  38.0
     2      49.0  36.7  45.0  35.5  30.0  38.4

In [85]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      53.0  36.3
     2      21.0  35.7
2014 1      42.0  36.2
     2      45.0  35.5

In [78]:
np.round(np.random.randn(4,6),1)

array([[-0.59,  0.35, -1.38,  0.4 ,  0.45,  0.9 ],
       [ 0.86, -0.31, -0.42,  1.23, -0.11, -0.69],
       [ 2.34,  1.75,  1.27,  0.35, -0.13, -0.87],
       [-0.58, -1.2 , -0.83, -0.17, -0.39, -0.86]])

In [86]:
# multiIndex的索引和切片
## 多重的索引的Series
pop

state       year
California  2000    33871559
            2010     8885522
New York    2000    17896542
            2010     4549132
Texas       2000    20587842
            2010     6445123
dtype: int64

In [87]:
pop['California',2000]

33871559

In [88]:
pop['California']

year
2000    33871559
2010     8885522
dtype: int64

In [89]:
pop.loc['California':'New York']

state       year
California  2000    33871559
            2010     8885522
New York    2000    17896542
            2010     4549132
dtype: int64

In [90]:
pop[:,2000]

state
California    33871559
New York      17896542
Texas         20587842
dtype: int64

In [91]:
pop[pop>22000000]

state       year
California  2000    33871559
dtype: int64

In [92]:
pop[['California','Texas']]

state       year
California  2000    33871559
            2010     8885522
Texas       2000    20587842
            2010     6445123
dtype: int64

### 多重索引的DataFrame

In [93]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.0  36.8  53.0  36.3  37.0  36.9
     2      50.0  38.1  21.0  35.7  48.0  37.0
2014 1      29.0  37.9  42.0  36.2  51.0  38.0
     2      49.0  36.7  45.0  35.5  30.0  38.4

In [95]:
health_data['Guido','HR']

year  visit
2013  1        53.0
      2        21.0
2014  1        42.0
      2        45.0
Name: (Guido, HR), dtype: float64

In [97]:
health_data.iloc[:2,:4]

subject      Bob       Guido      
type          HR  Temp    HR  Temp
year visit                        
2013 1      42.0  36.8  53.0  36.3
     2      50.0  38.1  21.0  35.7

In [98]:
health_data.loc[:,('Bob','HR')]

year  visit
2013  1        42.0
      2        50.0
2014  1        29.0
      2        49.0
Name: (Bob, HR), dtype: float64

In [101]:
idx = pd.IndexSlice # Create an object to more easily perform multi-index slicing.
health_data.loc[idx[:,1],idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,42.0,53.0,37.0
2014,1,29.0,42.0,51.0


### 重排列多重索引
#### 已排序和未排序的索引
如果索引是為排序的，許多MultiIndex的切片操作將會失敗

In [104]:
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char','int']
data

char  int
a     1      0.170267
      2      0.246971
c     1      0.681248
      2      0.476066
b     1      0.047329
      2      0.632139
dtype: float64

In [106]:
#會出錯，因為未排序
#data['a':'b']

In [107]:
data = data.sort_index()
data

char  int
a     1      0.170267
      2      0.246971
b     1      0.047329
      2      0.632139
c     1      0.681248
      2      0.476066
dtype: float64

In [108]:
data['a':'b']

char  int
a     1      0.170267
      2      0.246971
b     1      0.047329
      2      0.632139
dtype: float64

### 索引的stacking和unstacking